In [ ]:
#!g1.1
import pandas as pd
import numpy as np

df = pd.read_csv("/home/jupyter/work/resources/CW_dataset/CW_dataset.csv")



In [79]:
#!g1.1

# split the text into word lists for training set
list_of_texts = []
for i in range(df.shape[0]):
    #pull the words
    words = str(df['text'][i])
    #remove the punctuation
    q = len(df['text'][i].split())
    list_of_texts.append(q)
   


In [80]:
#!g1.1
import statistics
print(statistics.mean(list_of_texts))

567.4438295517432


In [160]:
#!g1.1
import transformers
import torch
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
#tokenizer = AutoTokenizer.from_pretrained('cointegrated/rubert-tiny2')





In [4]:
#!g1.1
import pandas as pd
import numpy as np
from datasets import load_dataset

dataset_train = load_dataset('/home/jupyter/work/resources/CW_dataset', data_files='CW_dataset.csv', split='train[:70%]')
dataset_eval = load_dataset('/home/jupyter/work/resources/CW_dataset', data_files='CW_dataset.csv', split='train[70%:90%]')
dataset_test = load_dataset('/home/jupyter/work/resources/CW_dataset', data_files='CW_dataset.csv', split='train[90%:]')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
#!g1.1
def preprocess_function(examples):
        return tokenizer(examples["text"], padding='max_length', max_length = 512, truncation=True)

In [6]:
#!g1.1
tokenized_CW_train = dataset_train.map(preprocess_function, batched=True)
tokenized_CW_eval = dataset_eval.map(preprocess_function, batched=True)
tokenized_CW_test = dataset_test.map(preprocess_function, batched=True)

Parameter 'function'=<function preprocess_function at 0x7ffaf0611ee0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/1265 [00:00<?, ? examples/s]

Map:   0%|          | 0/361 [00:00<?, ? examples/s]

Map:   0%|          | 0/181 [00:00<?, ? examples/s]

In [7]:
#!g1.1
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [8]:
#!g1.1
%pip install evaluate

import evaluate

accuracy = evaluate.load("accuracy")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


In [9]:
#!g1.1
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [10]:
#!g1.1
id2label = {0: "Bolsheviki", 1: "Mensheviki", 2: "Left_SRs", 3: "Kadets", 4: "Rights", 5: "White_army", 6: "Work_oppose", 7: "Nobles"}
label2id = {"Bolsheviki": 0, "Mensheviki": 1,"Left_SRs": 2, "Kadets": 3,"Rights": 4, "White_army": 5,"Work_oppose": 6, "Nobles": 7}

In [100]:
#!g1.1
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "DeepPavlov/rubert-base-cased", num_labels=8, id2label=id2label, label2id=label2id
)

# model = AutoModelForSequenceClassification.from_pretrained(
#     "cointegrated/rubert-tiny2", num_labels=8, id2label=id2label, label2id=label2id
# )



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [101]:
#!g1.1
training_args = TrainingArguments(
    output_dir="/home/jupyter/datasphere/project/Trained_Bert_classifier_2",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to='none'
)

Failed to deserialize variable 'training_args'. Run the following code to delete it:
  del_datasphere_variables('accuracy', 'data_collator', 'dataset_eval', 'dataset_test', 'dataset_train', 'tokenized_CW_eval', 'tokenized_CW_train', 'tokenizer', 'training_args')
Traceback (most recent call last):
  File "/kernel/lib/python3.9/site-packages/ml_kernel/state/state_protocol.py", line 283, in _load_component
    value = unpickler.load()
  File "/home/jupyter/.local/lib/python3.9/site-packages/datasets/table.py", line 1069, in __setstate__
    table = _memory_mapped_arrow_table_from_file(path)
  File "/home/jupyter/.local/lib/python3.9/site-packages/datasets/table.py", line 65, in _memory_mapped_arrow_table_from_file
    opened_stream = _memory_mapped_record_batch_reader_from_file(filename)
  File "/home/jupyter/.local/lib/python3.9/site-packages/datasets/table.py", line 50, in _memory_mapped_record_batch_reader_from_file
    memory_mapped_stream = pa.memory_map(filename)
  File "pyarrow/io.

In [102]:
#!g1.1
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_CW_train,
    eval_dataset=tokenized_CW_eval,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

Failed to deserialize variable 'training_args'. Run the following code to delete it:
  del_datasphere_variables('accuracy', 'data_collator', 'dataset_eval', 'dataset_test', 'dataset_train', 'tokenized_CW_eval', 'tokenized_CW_train', 'tokenizer', 'training_args')
Traceback (most recent call last):
  File "/kernel/lib/python3.9/site-packages/ml_kernel/state/state_protocol.py", line 283, in _load_component
    value = unpickler.load()
  File "/home/jupyter/.local/lib/python3.9/site-packages/datasets/table.py", line 1069, in __setstate__
    table = _memory_mapped_arrow_table_from_file(path)
  File "/home/jupyter/.local/lib/python3.9/site-packages/datasets/table.py", line 65, in _memory_mapped_arrow_table_from_file
    opened_stream = _memory_mapped_record_batch_reader_from_file(filename)
  File "/home/jupyter/.local/lib/python3.9/site-packages/datasets/table.py", line 50, in _memory_mapped_record_batch_reader_from_file
    memory_mapped_stream = pa.memory_map(filename)
  File "pyarrow/io.

In [103]:
#!g1.1
trainer.train()

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss


...

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.166293,0.958449


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Failed to deserialize variable 'accuracy'. Run the following code to delete it:
  del_datasphere_variables('accuracy', 'data_collator', 'dataset_eval', 'dataset_test', 'dataset_train', 'tokenized_CW_eval', 'tokenized_CW_train', 'tokenizer', 'training_args')
Traceback (most recent call last):
  File "/kernel/lib/python3.9/site-packages/ml_kernel/state/state_protocol.py", line 283, in _load_component
    value = unpickler.load()
  File "/home/jupyter/.local/lib/python3.9/site-packages/datasets/table.py", line 1069, in __setstate__
    table = _memory_mapped_arrow_table_from_file(path)
  File "/home/jupyter/.local/lib/python3.9/site-packages/datasets/table.py", line 65, in _memory_mapped_arrow_table_from_file
    opened_stream = _memory_mapped_record_batch_reader_fro

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.166293,0.958449
2,0.632600,0.118992,0.963989


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.166293,0.958449
2,0.632600,0.118992,0.963989


TrainOutput(global_step=634, training_loss=0.5288959978506769, metrics={'train_runtime': 183.1342, 'train_samples_per_second': 13.815, 'train_steps_per_second': 3.462, 'total_flos': 665706830807040.0, 'train_loss': 0.5288959978506769, 'epoch': 2.0})